In [1]:
import os
os.environ["DJANGO_ALLOW_ASYNC_UNSAFE"] = "true"

In [8]:
from project.models import Project
from public_data.models import OcsgeDiff

from django.contrib.gis.db.models.functions import Intersection, Area, Transform
from django.db.models import Sum, F, Value, Q, Min, Max
from django.db.models import Case, Value, When, DecimalField
from django.db.models.functions import Cast

from prettytable import PrettyTable

In [3]:
project = Project.objects.get(id=13)
project

<Project: Diagnostic de CA Grand Auch Coeur de Gascogne>

In [11]:
geom = project.combined_emprise
qs = OcsgeDiff.objects.all()
# sélection
qs = qs.filter(year_old__gte=project.analyse_start_date, year_new__lte=project.analyse_end_date)
qs = qs.filter(Q(is_new_artif=True) | Q(is_new_natural=True))
qs = qs.annotate(intersection=Transform(Intersection("mpoly", geom), 2154))
qs = qs.annotate(intersection_area=Area("intersection"))
qs = qs.filter(mpoly__intersects=geom)
qs = qs.annotate(code_prefix=Case(
    When(is_new_artif=True, then=F("new_matrix__couverture__code_prefix")),
    default=F("old_matrix__couverture__code_prefix"),
))
qs = qs.values("code_prefix")
qs = qs.annotate(
    artif=Cast(
        Sum("intersection_area", filter=Q(is_new_artif=True), default=0) / 10000,
        DecimalField(max_digits=15, decimal_places=2)
    ),
    renat=Cast(
        Sum("intersection_area", filter=Q(is_new_natural=True), default=0) / 10000,
        DecimalField(max_digits=15, decimal_places=2)
    ),
)
qs

<QuerySet [{'code_prefix': 'CS1.1.1.1', 'artif': Decimal('7.47'), 'renat': Decimal('0.97')}, {'code_prefix': 'CS1.1.1.2', 'artif': Decimal('8.47'), 'renat': Decimal('0.01')}, {'code_prefix': 'CS1.1.2.1', 'artif': Decimal('24.84'), 'renat': Decimal('2.87')}, {'code_prefix': 'CS2.2.1', 'artif': Decimal('28.95'), 'renat': Decimal('1.88')}]>

In [40]:
data = {
    line["code_prefix"]: {"artif": line["artif"], "renat": line["renat"]}
    for line in qs
}
data

{'CS1.1.1.1': {'artif': Decimal('7.47'), 'renat': Decimal('0.97')},
 'CS1.1.1.2': {'artif': Decimal('8.47'), 'renat': Decimal('0.01')},
 'CS1.1.2.1': {'artif': Decimal('24.84'), 'renat': Decimal('2.87')},
 'CS2.2.1': {'artif': Decimal('28.95'), 'renat': Decimal('1.88')}}

In [26]:
qs2 = CommuneSol.objects.filter(
    city__in=project.cities.all(), year__in=[project.first_year_ocsge, project.last_year_ocsge]
)
qs2 = qs2.filter(matrix__is_artificial=True)
qs2 = qs2.annotate(code_prefix=F("matrix__couverture__code_prefix"))
qs2 = qs2.annotate(label=F("matrix__couverture__label"))
qs2 = qs2.annotate(label_short=F("matrix__couverture__label_short"))
qs2 = qs2.annotate(is_artif=F("matrix__is_artificial"))
qs2 = qs2.values("code_prefix", "label", "label_short")
qs2 = qs2.annotate(
    surface_first=
    Cast(
        Sum("surface", filter=Q(year=project.first_year_ocsge), default=0),
        DecimalField(max_digits=15, decimal_places=2)
    ),
    surface_last=Cast(
        Sum("surface", filter=Q(year=project.last_year_ocsge), default=0),
        DecimalField(max_digits=15, decimal_places=2)
    ),
    surface_diff=F("surface_last") - F("surface_first"),
)
qs2

<QuerySet [{'code_prefix': 'CS1.1.1.1', 'label': 'Zones bâties', 'label_short': None, 'surface_first': Decimal('748.06'), 'surface_last': Decimal('763.14'), 'surface_diff': Decimal('15.08')}, {'code_prefix': 'CS1.1.1.2', 'label': 'Zones non bâties (Routes; places; parking…)', 'label_short': None, 'surface_first': Decimal('1261.06'), 'surface_last': Decimal('1271.82'), 'surface_diff': Decimal('10.76')}, {'code_prefix': 'CS1.1.2.1', 'label': 'Zones à matériaux minéraux', 'label_short': None, 'surface_first': Decimal('145.48'), 'surface_last': Decimal('167.11'), 'surface_diff': Decimal('21.63')}, {'code_prefix': 'CS2.2.1', 'label': 'Formations herbacées (Pelouses et prairies; terres arables; roselières; …)', 'label_short': None, 'surface_first': Decimal('2787.08'), 'surface_last': Decimal('2803.61'), 'surface_diff': Decimal('16.53')}]>

In [47]:
x = PrettyTable(field_names=["code_prefix", "label", "artif", "% artif", "renat", "% renat", "last", "% surface"], align="l")

results = list(qs2)

total_surface = total_artif = total_renat = 0

for result in results:
    code_prefix = result["code_prefix"]
    result["artif"] = data[code_prefix]["artif"]
    result["renat"] = data[code_prefix]["renat"]
    total_surface += result["surface_last"]
    total_artif += data[code_prefix]["artif"]
    total_renat += data[code_prefix]["renat"]

for result in results:
    line = (
        result["code_prefix"],
        result["label"][:30],
        result["artif"],
        round(100 * result["artif"] / total_artif, 0),
        result["renat"],
        round(100 * result["renat"] / total_renat, 0),
        result["surface_last"],
        round(100 * result["surface_last"] / total_surface, 0),
    )
    x.add_row(line)
print("Progression de l'artif")
print(project.name)
print(x)

Progression de l'artif
Diagnostic de CA Grand Auch Coeur de Gascogne
+-------------+--------------------------------+-------+---------+-------+---------+---------+-----------+
| code_prefix | label                          | artif | % artif | renat | % renat | last    | % surface |
+-------------+--------------------------------+-------+---------+-------+---------+---------+-----------+
| CS1.1.1.1   | Zones bâties                   | 7.47  | 11      | 0.97  | 17      | 763.14  | 15        |
| CS1.1.1.2   | Zones non bâties (Routes; plac | 8.47  | 12      | 0.01  | 0       | 1271.82 | 25        |
| CS1.1.2.1   | Zones à matériaux minéraux     | 24.84 | 36      | 2.87  | 50      | 167.11  | 3         |
| CS2.2.1     | Formations herbacées (Pelouses | 28.95 | 42      | 1.88  | 33      | 2803.61 | 56        |
+-------------+--------------------------------+-------+---------+-------+---------+---------+-----------+
